In [1]:
# data managing and display libs
import pandas as pd
import numpy as np
import os
import io

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline 

# sagemaker libraries
import boto3

In [16]:
df_airbnb = pd.read_csv('airbnb_clean.csv')

In [17]:
drop_cols = list(df_airbnb.select_dtypes(['O']).columns) + ['id', 'latitude', 'longitude', 'maximum_nights']
drop_cols

['name',
 'summary',
 'description',
 'host_verifications',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'amenities',
 'id',
 'latitude',
 'longitude',
 'maximum_nights']

In [18]:
df_airbnb.drop(drop_cols, axis=1, inplace=True)

In [19]:
# convert all these to float64
df_airbnb = df_airbnb.astype(float)

In [20]:
df_airbnb.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45605 entries, 0 to 45604
Data columns (total 56 columns):
host_listings_count                                45605 non-null float64
host_total_listings_count                          45605 non-null float64
accommodates                                       45605 non-null float64
bathrooms                                          45605 non-null float64
bedrooms                                           45605 non-null float64
beds                                               45605 non-null float64
price                                              45605 non-null float64
guests_included                                    45605 non-null float64
extra_people                                       45605 non-null float64
minimum_nights                                     45605 non-null float64
availability_30                                    45605 non-null float64
availability_60                                    45605 non-null float64
ava

## Deal with outliers

In [21]:
# Check if each value of mySeries lies within the outlier range.
# If the value is greater than upperOutlier mark it True
# If the value is lower than lowerOutlier mark it True
# return the total number of outliers for that series.
def numberOfOutliers(mySeries, upperOutlier, lowerOutlier):
    return sum((mySeries > upperOutlier.loc[mySeries.name, ]) |\
               (mySeries < lowerOutlier.loc[mySeries.name,]))

def outlier_stats(df):
    numericDescribe = (df.describe(include='all').T).round(decimals=3)
    
    # Calculate outliers using this formula: first quartile – 1.5·IQR > outlier > third quartile + 1.5·IQR
    numericDescribe['IQR'] = numericDescribe['75%'] - numericDescribe['25%']
    numericDescribe['outliers'] = (numericDescribe['max'] > (numericDescribe['75%'] + (1.5 * numericDescribe['IQR']))) \
                            | (numericDescribe['min'] < (numericDescribe['25%'] - (1.5 * numericDescribe['IQR'])))
    
    # Calculate IQR for each column of the dataframe.
    IQR = df.quantile(.75) - df.quantile(.25)
    
    # Calculate the upper and lower outlier values
    upperOutlier = df.quantile(.75) + (1.5 * (IQR))
    lowerOutlier = df.quantile(.25) - (1.5 * (IQR))
    
    # Store the result in a new column
    numericDescribe['num_outliers'] = df.apply(numberOfOutliers, args=(upperOutlier, lowerOutlier))
    numericDescribe.sort_values('num_outliers', ascending=False, inplace=True)
    newColOrder = ['count', 'outliers', 'num_outliers', 'IQR', 'mean', 'std', \
               'min', '25%', '50%', '75%', 'max']
    numericDescribe = numericDescribe.reindex(columns=newColOrder)
    
    return numericDescribe

In [22]:
outlier_stats_df = outlier_stats(df_airbnb)
outlier_stats_df

,count,outliers,num_outliers,IQR,mean,std,min,25%,50%,75%,max
bedrooms,45605.0,True,12668,0.0,1.179,0.741,0.0,1.0,1.0,1.0,14.0
cancellation_policy_moderate,45605.0,True,10558,0.0,0.232,0.422,0.0,0.0,0.0,0.0,1.0
host_is_superhost_f,45605.0,True,8752,0.0,0.808,0.394,0.0,1.0,1.0,1.0,1.0
host_is_superhost_t,45605.0,True,8752,0.0,0.192,0.394,0.0,0.0,0.0,0.0,1.0
is_location_exact_t,45605.0,True,7887,0.0,0.827,0.378,0.0,1.0,1.0,1.0,1.0
is_location_exact_f,45605.0,True,7887,0.0,0.173,0.378,0.0,0.0,0.0,0.0,1.0
property_type_Apartment,45605.0,True,7000,0.0,0.847,0.360,0.0,1.0,1.0,1.0,1.0
bathrooms,45605.0,True,6949,0.0,1.143,0.428,0.0,1.0,1.0,1.0,15.5
host_total_listings_count,45605.0,True,6858,1.0,14.828,90.301,0.0,1.0,1.0,2.0,1465.0
host_listings_count,45605.0,True,6858,1.0,14.828,90.301,0.0,1.0,1.0,2.0,1465.0


## Remove outliers from offending columns

In [23]:
df_airbnb['price_adjusted'] = df_airbnb['price'] / df_airbnb['minimum_nights']

In [24]:
df_airbnb.drop(['price', 'minimum_nights'], axis='columns', inplace=True)

In [25]:
drop_more_cols = ['host_total_listings_count',
                  'host_listings_count',
                 'host_total_listings_count',
                 'calculated_host_listings_count_entire_homes',
                 'calculated_host_listings_count_private_rooms',
                 'availability_365']

In [26]:
df_airbnb.drop(drop_more_cols, axis=1, inplace=True)

In [27]:
outlier_stats(df_airbnb)

,count,outliers,num_outliers,IQR,mean,std,min,25%,50%,75%,max
bedrooms,45605.0,True,12668,0.0,1.179,0.741,0.0,1.0,1.0,1.0,14.0
cancellation_policy_moderate,45605.0,True,10558,0.0,0.232,0.422,0.0,0.0,0.0,0.0,1.0
host_is_superhost_t,45605.0,True,8752,0.0,0.192,0.394,0.0,0.0,0.0,0.0,1.0
host_is_superhost_f,45605.0,True,8752,0.0,0.808,0.394,0.0,1.0,1.0,1.0,1.0
is_location_exact_f,45605.0,True,7887,0.0,0.173,0.378,0.0,0.0,0.0,0.0,1.0
is_location_exact_t,45605.0,True,7887,0.0,0.827,0.378,0.0,1.0,1.0,1.0,1.0
property_type_Apartment,45605.0,True,7000,0.0,0.847,0.360,0.0,1.0,1.0,1.0,1.0
bathrooms,45605.0,True,6949,0.0,1.143,0.428,0.0,1.0,1.0,1.0,15.5
calculated_host_listings_count,45605.0,True,6034,1.0,7.317,35.596,1.0,1.0,1.0,2.0,343.0
number_of_reviews,45605.0,True,5618,23.0,23.397,45.101,0.0,1.0,5.0,24.0,639.0


In [28]:
df=df_airbnb.copy()

## Use StandardScaler

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
standard_scaler = StandardScaler()

In [31]:
scaled_df = pd.DataFrame(standard_scaler.fit_transform(df), columns=df.columns)

In [33]:
outlier_stats(scaled_df)

,count,outliers,num_outliers,IQR,mean,std,min,25%,50%,75%,max
bedrooms,45605.0,True,12668,0.000,0.0,1.0,-1.591,-0.241,-0.241,-0.241,17.313
cancellation_policy_moderate,45605.0,True,10558,0.000,-0.0,1.0,-0.549,-0.549,-0.549,-0.549,1.822
host_is_superhost_t,45605.0,True,8752,0.000,-0.0,1.0,-0.487,-0.487,-0.487,-0.487,2.052
host_is_superhost_f,45605.0,True,8752,0.000,0.0,1.0,-2.052,0.487,0.487,0.487,0.487
is_location_exact_f,45605.0,True,7887,0.000,0.0,1.0,-0.457,-0.457,-0.457,-0.457,2.187
is_location_exact_t,45605.0,True,7887,0.000,-0.0,1.0,-2.187,0.457,0.457,0.457,0.457
property_type_Apartment,45605.0,True,7000,0.000,-0.0,1.0,-2.348,0.426,0.426,0.426,0.426
bathrooms,45605.0,True,6949,0.000,0.0,1.0,-2.667,-0.333,-0.333,-0.333,33.511
calculated_host_listings_count,45605.0,True,6034,0.028,-0.0,1.0,-0.177,-0.177,-0.177,-0.149,9.431
number_of_reviews,45605.0,True,5618,0.510,0.0,1.0,-0.519,-0.497,-0.408,0.013,13.650


## Use MinMaxScaler

In [37]:
from sklearn.preprocessing import MinMaxScaler

In [38]:
min_max_scaler = MinMaxScaler()

In [39]:
min_max_scaled_df = pd.DataFrame(min_max_scaler.fit_transform(df), columns=df.columns)

In [40]:
outlier_stats(min_max_scaled_df)

,count,outliers,num_outliers,IQR,mean,std,min,25%,50%,75%,max
bedrooms,45605.0,True,12668,0.000,0.084,0.053,0.0,0.071,0.071,0.071,1.0
cancellation_policy_moderate,45605.0,True,10558,0.000,0.232,0.422,0.0,0.000,0.000,0.000,1.0
host_is_superhost_f,45605.0,True,8752,0.000,0.808,0.394,0.0,1.000,1.000,1.000,1.0
host_is_superhost_t,45605.0,True,8752,0.000,0.192,0.394,0.0,0.000,0.000,0.000,1.0
is_location_exact_f,45605.0,True,7887,0.000,0.173,0.378,0.0,0.000,0.000,0.000,1.0
is_location_exact_t,45605.0,True,7887,0.000,0.827,0.378,0.0,1.000,1.000,1.000,1.0
property_type_Apartment,45605.0,True,7000,0.000,0.847,0.360,0.0,1.000,1.000,1.000,1.0
bathrooms,45605.0,True,6949,0.000,0.074,0.028,0.0,0.065,0.065,0.065,1.0
calculated_host_listings_count,45605.0,True,6034,0.003,0.018,0.104,0.0,0.000,0.000,0.003,1.0
number_of_reviews,45605.0,True,5618,0.036,0.037,0.071,0.0,0.002,0.008,0.038,1.0


## Get the merged_df

In [34]:
merged_df = pd.read_csv('merged_df.csv')

In [35]:
merged_df.head()

,amenities_tv,amenities_wifi,amenities_air_conditioning,amenities_kitchen,amenities_paid_parking_off_premises,amenities_free_street_parking,amenities_indoor_fireplace,amenities_heating,amenities_family/kid_friendly,amenities_smoke_detector,...,description_contains_yankee,description_contains_yard,description_contains_year,description_contains_yellow,description_contains_yoga,description_contains_york,description_contains_young,description_contains_yummy,description_contains_zero,description_contains_zone
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.00000,0.0,0.137645,0.000000,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.00000,0.0,0.000000,0.074083,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.18384,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [36]:
outlier_stats(merged_df)

,count,outliers,num_outliers,IQR,mean,std,min,25%,50%,75%,max
description_contains_home,45605.0,True,11384,0.0,0.026,0.054,0.0,0.0,0.0,0.0,0.916
description_contains_place,45605.0,True,11317,0.0,0.030,0.069,0.0,0.0,0.0,0.0,1.000
description_contains_city,45605.0,True,11249,0.0,0.024,0.048,0.0,0.0,0.0,0.0,0.596
host_verification_by_selfie,45605.0,True,11126,0.0,0.244,0.429,0.0,0.0,0.0,0.0,1.000
amenities_coffee_maker,45605.0,True,11087,0.0,0.243,0.429,0.0,0.0,0.0,0.0,1.000
description_contains_quiet,45605.0,True,11035,0.0,0.024,0.048,0.0,0.0,0.0,0.0,1.000
host_verification_by_kba,45605.0,True,10985,0.0,0.241,0.428,0.0,0.0,0.0,0.0,1.000
description_contains_brooklyn,45605.0,True,10794,0.0,0.029,0.063,0.0,0.0,0.0,0.0,0.679
amenities_cable_tv,45605.0,True,10706,0.0,0.235,0.424,0.0,0.0,0.0,0.0,1.000
description_contains_large,45605.0,True,10567,0.0,0.025,0.055,0.0,0.0,0.0,0.0,0.762


## Concatenate both

In [41]:
final_df = pd.concat([min_max_scaled_df, merged_df], axis='columns')

In [42]:
final_df.shape

(45605, 2201)

In [43]:
final_df.head() #<------- THIS IS THE ONE I AM MERGING WITH TEXT FEATURES

,accommodates,bathrooms,bedrooms,beds,guests_included,extra_people,availability_30,availability_60,availability_90,number_of_reviews,...,description_contains_yankee,description_contains_yard,description_contains_year,description_contains_yellow,description_contains_yoga,description_contains_york,description_contains_young,description_contains_yummy,description_contains_zero,description_contains_zone
0,0.04,0.064516,0.000000,0.025,0.066667,0.000000,0.433333,0.283333,0.344444,0.071987,...,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.04,0.064516,0.071429,0.025,0.066667,0.066667,1.000000,1.000000,1.000000,0.000000,...,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.00,0.064516,0.071429,0.025,0.000000,0.066667,0.000000,0.000000,0.000000,0.014085,...,0.0,0.0,0.00000,0.0,0.137645,0.000000,0.0,0.0,0.0,0.0
3,0.04,0.064516,0.071429,0.025,0.066667,0.333333,0.800000,0.550000,0.700000,0.117371,...,0.0,0.0,0.00000,0.0,0.000000,0.074083,0.0,0.0,0.0,0.0
4,0.04,0.064516,0.071429,0.025,0.000000,0.100000,0.000000,0.000000,0.000000,0.076682,...,0.0,0.0,0.18384,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


## Use RobustScaler
Since each of the features have different units of measurement, we will use a RobustScaler. The MinMaxScaler is suseptible to outliers. We have seen that price has many outliers, so for that reason we will go with RobustScaler.

In [37]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaled_final_df = scaler.fit_transform(final_df)
scaled_final_df = pd.DataFrame(scaled_final_df, columns=final_df.columns)

scaled_final_df.head()

,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,...,description_contains_yankee,description_contains_yard,description_contains_year,description_contains_yellow,description_contains_yoga,description_contains_york,description_contains_young,description_contains_yummy,description_contains_zero,description_contains_zone
0,4.0,4.0,0.0,0.0,-1.0,0.0,1.157407,1.0,0.0,-0.50,...,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.462963,1.0,0.8,0.00,...,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,-0.5,0.0,0.0,0.0,-0.185185,0.0,0.8,1.75,...,0.0,0.0,0.00000,0.0,0.137645,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.925926,1.0,4.0,0.00,...,0.0,0.0,0.00000,0.0,0.000000,0.074083,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,-0.370370,0.0,1.2,10.50,...,0.0,0.0,0.18384,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


## Upload the minmax scaled df that was merged with merged_df

In [44]:
final_df.to_csv('min_max_scaled_final_df.csv', index=False)

In [45]:
# boto3 client to get S3 data
s3_client = boto3.client('s3')
bucket_name='skuchkula-sagemaker-airbnb'

In [46]:
# upload it to S3
s3_client.upload_file(Bucket=bucket_name, 
                      Filename='min_max_scaled_final_df.csv', 
                      Key='feature_eng/min_max_scaled_final_df.csv')